In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *
from bitstamp import *

from tqdm.notebook import tqdm

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from pytorch_tabnet.tab_model import TabNetClassifier

from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

In [3]:
load_from_disk = True

# '2Min'

In [4]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [5]:
class TabNetClassifierEarly(TabNetClassifier):
    def fit(self, X_train, Y_train):
        super().fit(X_train, 
                    Y_train, 
                    eval_set=[(X_train, Y_train)],
                    eval_metric=['balanced_accuracy'])

In [6]:
%%time

def get_classifiers():
    clss = classifiers = [
        lambda : CatBoostClassifier(logging_level = 'Silent'),
        
        #lambda : make_pipeline(StandardScaler(),
        #                        MLPClassifier(
        #                            solver='lbfgs', alpha=1, random_state=1, max_iter=20000,
        #                            early_stopping=True)
        #                        ),
        
        #lambda : MLPClassifier(alpha=1, random_state=1, max_iter=20000, early_stopping=True),
        
        lambda : xgb.XGBClassifier(random_state=1,
                                   objective = "binary:logistic",
                                   eval_metric='mlogloss',
                                   learning_rate=0.01),
        
        #lambda : GaussianNB(),
        #lambda : QuadraticDiscriminantAnalysis(),
        #lambda : AdaBoostClassifier(random_state = 42),
        
        lambda : TabNetClassifierEarly(verbose=0),
        
        #lambda : KNeighborsClassifier(3),
        #lambda : GaussianProcessClassifier(1.0 * RBF(1.0)),
        
        
        lambda : DecisionTreeClassifier(random_state = 42),
        #lambda : RandomForestClassifier(max_depth=50, n_estimators=100, max_features=1),
        lambda : rand(),
        
        #lambda : SVC(gamma=2, C=1, random_state = 42),
    ]
    
    def rand():
        ##print("rand created")
        param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
        return RandomForestClassifier(**param)
    

    clss = []
    clss.append(lambda : rand())
    #clss.append(lambda : make_pipeline(StandardScaler(),TabNetClassifierEarly(verbose=0)))
    return clss

estimators = [
                lambda : RFE(estimator=RandomForestClassifier(random_state=10), n_features_to_select=10),
                lambda : RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=8),
                lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10)),
                lambda : RFECV(estimator=RandomForestClassifier(random_state=10))
                ]


scoring = [
    'accuracy', 
    'f1', 
    'precision', 
    'recall'
]

estimators = []

#for score in scoring:
    
estimators.append(lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10),
                    scoring='accuracy'))    
estimators.append(lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10),
                    scoring='f1'))    
estimators.append(lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10),
                    scoring='precision'))    
estimators = []
estimators.append(lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10),
                    scoring='recall'))

def add_normalizers(models, cls):
    models.append(make_pipeline(StandardScaler(),cls()))
    models.append(make_pipeline(MinMaxScaler(),cls()))
    models.append(make_pipeline(Normalizer(),cls()))


def get_all_models():
    models = []
    classifiers = get_classifiers()
    for cls in classifiers:
        add_normalizers(models, cls)
        #models.append(MockModel(cls()))
        #models.append(MockCoPilotModel(cls(), getModel()))
        
        for est in estimators:
            tmp = lambda : Pipeline(
                steps=[
                ('s',est()),
                ('m',cls())
            ])
            add_normalizers(models, tmp)
            #models.append(tmp())
            #models.append(MockModel(tmp()))
            #models.append(MockCoPilotModel(tmp(), getModel()))
    return models


CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 11 µs


In [7]:
get_all_models_factory = lambda : get_all_models()

In [8]:
%%time

def test_models(provider, get_all_models_factory, steps = [1]):
    
    score_board_by_step = {}

    for step in steps:
        print(f"Step {step}")
        models_list = get_all_models_factory()
        model_result = []
        for idx in tqdm(range(len(models_list))):
            model_creator = models_list[idx]
            model = model_creator
            #print(f"Trainning {model}")
            train_by_step(model, step, provider)
            
            result = run_trial(model, provider, step)

            result['step'] = step

            model_result.append(result)
        
        score_board_by_step[step] = model_result
        
    best_models = []

    best_model = []

    best_score = -1
    
    best_step = -1
    
    model_rank = []
    
    def myFunc(e):
        return e['profit']

    for step_idx in score_board_by_step:
        step_board = score_board_by_step[step_idx]
        for result in step_board:
            if (result['profit'] > 0):
                model_rank.append(result)
            
    model_rank.sort(key=myFunc, reverse = True)
    
    return model_rank

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 9.06 µs


In [9]:

steps = [1, 5]

windows = [5, 10, 20, 30, 40]
#windows = [40]

minutes = [15, 5, 3, 1]
#minutes = [3]


In [10]:
def process_model(minu, win, step, model, provider):
        #print(f"Training {model} {provider} ")
        train_by_step(model, step, provider)
        #print(f"Checking {minu} {win} {step} ")
        result = run_trial(model, provider, step)
        result['window'] = win
        result['minutes'] = minu
        result['step'] = step
        #print(result)
        return result

In [11]:
%%time

min_results = []

process_list = []

for minu in minutes:
    results = []
    for win in windows:
        tec = TecAn(windows = win, windows_limit = 100)
        sourceDataGenerator = SourceDataGenerator(tec = tec)
        provider = get_online_data(minu, sourceDataGenerator, load_from_disk, win)
        for step in steps:
            models_list = get_all_models_factory()
            for idx in range(len(models_list)):
                model_creator = models_list[idx]
                model = model_creator
                process = {}
                process['window'] = win
                process['minutes'] = minu
                process['step'] = step
                process['model'] = model
                process['provider'] = provider
                process_list.append(process)

for process in tqdm(process_list):
    win = process['window'] 
    minu = process['minutes']
    step = process['step'] 
    model = process['model'] 
    provider = process['provider'] 
    
    tec = TecAn(windows = win, windows_limit = 100)
    sourceDataGenerator = SourceDataGenerator(tec = tec)
    provider = get_online_data(minu, sourceDataGenerator, load_from_disk, win)
    
    result = process_model(minu, win, step, model, provider)
    min_results.append(result)

  0%|          | 0/240 [00:00<?, ?it/s]

CPU times: user 4h 23min 34s, sys: 2min 13s, total: 4h 25min 48s
Wall time: 4h 31min 45s


In [12]:
model_path = f'model/all_results'
dump(min_results, model_path)

['model/all_results']

In [13]:
best_results = []
for win_result in min_results:
    profit = win_result['profit']
    model = win_result['model']
    window = win_result['window']
    minutes = win_result['minutes']
    step = win_result['step']
    profit = win_result['profit']
    models_score = win_result['models_score']
    #print(models_score)
    if (profit > 100):
        print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
        print(f"{win_result['model']}")
        print(f"")
        best_results.append(win_result)

print(f"{len(best_results)} selected")

Minutes=15 Window=5 Step=1 | 100.01436751411367
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=15 Window=5 Step=1 | 100.01436751411367
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=15 Window=5 Step=1 | 100.754

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=3 Window=5 Step=1 | 100.71844560982993
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                

                                        n_estimators=30, random_state=42))])

Minutes=1 Window=40 Step=1 | 100.34355313061027
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=1 Window=40 Step=1 | 100.18498723338602
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),


In [14]:
model_path = f'model/results'
dump(best_results, model_path)

['model/results']

In [15]:
def print_result(result):
    model = result['model']
    window = result['window']
    minutes = result['minutes']
    step = result['step']
    profit = result['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit} {best['models_profit_metric']}")
    print(f"{model}")

In [16]:
def myFunc(e):
    return e['profit']
    #return e['models_profit_metric']['btcusd']

best_results.sort(key=myFunc, reverse = False)

for best in best_results:
    print_result(best)
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    model_path = f'model/result_{window}_{minutes}_{step}'
    dump(best, model_path)
    print(f"")

Minutes=3 Window=30 Step=1 | 100.00342848896511 {'ltceur': 0.015279344278762323, 'ltcusd': -0.0454292269280058, 'ethusd': 0.036423002123218015, 'btcusd': -0.014182816584286384}
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=1 Window=20 Step=1 | 100.0086962907278 {'ltceur': 0.008374843340818325, 'ltcusd': 0.02110218580733093, 'ethusd': -0.0007733265887955042, 'btcusd': -0.001163964623951637}
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=1 Window=10 Step=1 | 100.01004333767136 {'ltceur': 0.0048796160565473205, 'ltcusd': -0.003003967430256649, 'ethusd': -0.0032005656936964617


Minutes=1 Window=30 Step=1 | 100.39951520248437 {'ltceur': 0.005204451306014886, 'ltcusd': -0.010257354237082354, 'ethusd': 0.008992569973476488, 'btcusd': 0.0032754479144750884}
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=3 Window=5 Step=1 | 100.40589582316468 {'ltceur': -0.0015198547403199338, 'ltcusd': 0.04712362062719833, 'ethusd': -0.017096365550341677, 'btcusd': -0.0017520909130441086}
Pipeline(steps=[('normalize


Minutes=15 Window=10 Step=1 | 101.56670281250578 {'ltceur': 0.011881923543923286, 'ltcusd': -0.001121360509961691, 'ethusd': -0.012536124452956899, 'btcusd': 0.017047325700780692}
Pipeline(steps=[('normalizer', Normalizer()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=5 Window=20 Step=1 | 101.75590348568932 {'ltceur': 0.006056577587106279, 'ltcusd': -0.014268415232537116, 'ethusd': 0.0032485316737775926, 'btcusd': 0.02910613329838027}
Pipeline(steps=[('normalizer', N


Minutes=15 Window=10 Step=1 | 108.86938817937694 {'ltceur': 0.021986742006755582, 'ltcusd': 0.038356519116210824, 'ethusd': 0.02119776591957815, 'btcusd': -0.022665903127910225}
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=5 Window=10 Step=1 | 109.30167581889944 {'ltceur': 0.0366018671615414, 'ltcusd': 0.035937506569735905, 'ethusd': 0.027739289746640066, 'btcusd': 0.0009477162608192251}
Pipeline(steps=[('normalizer', Normalizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=5 Window=10 Step=1 | 109.30167581889944 {'ltceur': 0.0366018671615414, 'ltcusd': 0.035937506569735905, 'ethusd': 0.027739289746640066, 'btcusd': 0.

In [17]:
def myFunc(e):
    return e['profit']
    #return e['models_profit_metric']['btcusd']

best_results.sort(key=myFunc, reverse = True)

In [18]:
best_results[0]

{'profit': 110.51658317764043,
 'model': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier',
                  RandomForestClassifier(criterion='entropy', max_depth=100,
                                         n_estimators=30, random_state=42))]),
 'models_profit': {'ltceur': 12.0434,
  'ltcusd': 11.8861,
  'ethusd': 21.611,
  'btcusd': -3.4741},
 'models_score': {'ltceur': {'recall': 0.2,
   'precision': 0.4171322160148976,
   'f1': 0.27036813518406766,
   'accuracy': 0.5806451612903226,
   'roc_auc': 0.5112308564946114},
  'ltcusd': {'recall': 0.19318181818181818,
   'precision': 0.40925925925925927,
   'f1': 0.26247030878859856,
   'accuracy': 0.5691987513007284,
   'roc_auc': 0.5048715301374875},
  'ethusd': {'recall': 0.22641509433962265,
   'precision': 0.45517241379310347,
   'f1': 0.302405498281787,
   'accuracy': 0.5775234131113424,
   'roc_auc': 0.5211865803672487},
  'btcusd': {'recall': 0.2050561797752809,
   'precision': 0.396

In [19]:
def load_online(minutes, window, val_end, currency = []):
    tec = TecAn(windows = window, windows_limit = 100)
    source_data_generator = SourceDataGenerator(tec = tec)

    start = val_end - (60 * 100 * minutes)
    end = val_end - (160 * minutes)

    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 40,
                 val_limit = 999,
                 val_keys = currency,
                 val_start = start,
                 val_end = end,
                 train_start_list = []
    )

    online.load_cache()
    return online

In [20]:
def backtest_model(model, x, closed_prices, back):
    
    for idx in range(len(x)):
        xx = np.array([x[idx]])
        #print(xx)
        yy = model.predict(xx)
        yy = yy[0]
        price = closed_prices[idx]
        #print(f'{idx} {yy} {price}')
        back.on_state(0, price)
        if(yy == 1):
            back.request_buy(price)
        else:
            back.request_sell(price)
    
    #print(f"Closing backtest_model {price} {back.current}")
    back.sell(price)
    #print(f"Closed backtest_model {back.current}")
    
    return back

In [21]:
def test_model(model, set_key, provider, step, verbose = True):
    valX, valY = provider.load_val_data(set_key)

    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    #print(len(x))
    preds = model.predict(x)

    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    f1 = f1_score(y, preds)
    accuracy = accuracy_score(y, preds)

    back = BackTest(value = 100, 
                    verbose = verbose, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    back = backtest_model(model, x, closed_prices, back)
    return back

In [22]:
from datetime import datetime


timestamp = int(datetime.timestamp((datetime.now())))



In [23]:
online_cache = {}

In [24]:
scoreboard = []

fails = []

currency = "ltcusd"
currency = "ethusd"
currency = "batusd"

currs = ["batusd", "ltcusd", "ethusd", "btcusd"]
currs = ["btcusd"]

#currs = online.val_keys

for best in tqdm(best_results):
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    profit = best['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
    cache_key = f"{minutes}-{window}"
    try:
        online = online_cache[cache_key]
    except :
        print(f"Not found {cache_key}")
        online = load_online(minutes = minutes, window = window, val_end = timestamp, currency = currs)
        online_cache[cache_key] = online
        
    profits = []
    backs = {}
    score = {}
    
    has_negative = False
    
    for train_set in currs:
        back = test_model(model, train_set, online, step, False)
        back_profit = back.get_profit()
        profits.append(back_profit)
        backs[train_set] = back
        print(f"{train_set} - {back_profit}")
        if (back_profit <= 0 and not has_negative):
            has_negative = True
        
    score['profit'] = np.average(profits)
    score['backs'] = backs
    score['result'] = best
    if (not has_negative):
        scoreboard.append(score)



  0%|          | 0/55 [00:00<?, ?it/s]

Minutes=15 Window=20 Step=1 | 110.51658317764043
Not found 15-20
https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=900&limit=1000&start=1627260946
Downloaded from 2021-07-26 01:45:00 to 2021-07-27 02:30:00 37072.12
Resample 15Min - TecAn ( windows 20, windows_limit 100 )
Processing 99 of btcusd


  0%|          | 0/99 [00:00<?, ?it/s]

btcusd - 2.1623
Minutes=15 Window=10 Step=1 | 109.43121445370366
Not found 15-10
https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=900&limit=1000&start=1627260946
Downloaded from 2021-07-26 01:45:00 to 2021-07-27 02:30:00 37072.12
Resample 15Min - TecAn ( windows 10, windows_limit 100 )
Processing 99 of btcusd


  0%|          | 0/99 [00:00<?, ?it/s]

...

btcusd - 0.0
Minutes=1 Window=5 Step=1 | 100.03379323265345


/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


btcusd - 0.0
Minutes=15 Window=5 Step=1 | 100.01436751411367
btcusd - -2.0947
Minutes=15 Window=5 Step=1 | 100.01436751411367
btcusd - -2.0947
Minutes=1 Window=10 Step=1 | 100.01004333767136
btcusd - -0.0356
Minutes=1 Window=20 Step=1 | 100.0086962907278
btcusd - -0.0356
Minutes=3 Window=30 Step=1 | 100.00342848896511
Not found 3-30
https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=180&limit=1000&start=1627332946
Downloaded from 2021-07-26 21:54:00 to 2021-07-27 02:48:00 37115.30
Resample 3Min - TecAn ( windows 30, windows_limit 100 )
Processing 98 of btcusd


  0%|          | 0/98 [00:00<?, ?it/s]

btcusd - 0.5973


In [25]:
profs = []

print(f"Selected: {len(scoreboard)}")

scoreboard.sort(key=myFunc, reverse = True)

for score in scoreboard:
    profit = score['profit']
    result = score['result']
    backs = score['backs']
    #print(f"{profit}")
    for back in backs:
        #backs[back].report()
        print(f"{back} -> {backs[back].get_profit()}")
    print_result(result)
    #print(result)
    profs.append(profit)




Selected: 21
btcusd -> 6.3617
Minutes=15 Window=40 Step=1 | 107.54102490786364 {'ltceur': 0.015279344278762323, 'ltcusd': -0.0454292269280058, 'ethusd': 0.036423002123218015, 'btcusd': -0.014182816584286384}
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])
btcusd -> 6.0496
Minutes=15 Window=30 Step=1 | 107.96955189926528 {'ltceur': 0.015279344278762323, 'ltcusd': -0.0454292269280058, 'ethusd': 0.036423002123218015, 'btcusd': -0.0

In [26]:
winner = scoreboard[0]['result']
print_result(winner)
model_path = f'model/winner'
dump(winner, model_path)
print()
print(f"{np.average(profs)} ")

Minutes=15 Window=40 Step=1 | 107.54102490786364 {'ltceur': 0.015279344278762323, 'ltcusd': -0.0454292269280058, 'ethusd': 0.036423002123218015, 'btcusd': -0.014182816584286384}
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

2.1941380952380953 


In [27]:
model = scoreboard[0]['result']['model']

In [28]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])